In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dataset import Dataset, log_fit_fn
from submodular_monotone_net import SubmodularMonotoneNet
from torch.utils.data import random_split, DataLoader
import torch

In [62]:
DATASET_SIZE = int(1e4)
SAMPLE_SIZE = int(1e4)
Z_SIZE = 10

dataset = Dataset(DATASET_SIZE, SAMPLE_SIZE, Z_SIZE, log_fit_fn)

In [63]:
dataset[0]

(tensor([True, True, True,  ..., True, True, True]), tensor(10.8051))

In [64]:
train_size = int(0.8 * len(dataset))  # 80% for training
test_size = len(dataset) - train_size  # Remaining for testing

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoader for training and testing
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [65]:
from submodular_monotone_net import SubmodularMonotoneNet
monotone_network = SubmodularMonotoneNet(10, 0.5, SAMPLE_SIZE, Z_SIZE, [1, 1000, 1000, 1], dataset)

In [66]:
monotone_network.input_size

10000

In [67]:
opt = torch.optim.Adam(
    params=monotone_network.parameters(), lr=1e-2
)

num_epochs = 100
criterion = torch.nn.MSELoss()

for epoch in range(num_epochs):
    monotone_network.train()  # Set the model to training mode
    running_loss = 0.0
    for inputs, labels in train_loader:
        opt.zero_grad()  # Zero the parameter gradients
        
        outputs = monotone_network(inputs.to(torch.float32))  # Forward pass
        
        loss = criterion(outputs, labels)  # Compute loss
        
        loss.backward()  # Backward pass
        opt.step()  # Update parameters
        
        running_loss += loss.item() * inputs.size(0)
        # monotone_network.clamp_weights()
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

Epoch 1/100, Loss: 51.7341
Epoch 2/100, Loss: 10.8151
Epoch 3/100, Loss: 0.1153
Epoch 4/100, Loss: 0.0809


KeyboardInterrupt: 

In [41]:
for batch_data, batch_targets in train_loader:
    random_index = torch.randint(0, batch_data.size(0), (1,)).item()
    sample_data = batch_data[random_index]
    sample_target = batch_targets[random_index]
    print(monotone_network(sample_data.unsqueeze(0).to(torch.float32)))
    break  # Stop after getting the first sample

print(f"Random Sample Data: {sample_data}")
print(f"Random Sample Target: {sample_target}")

tensor(9.8117, grad_fn=<SqueezeBackward0>)
Random Sample Data: tensor([ True, False, False,  ..., False, False, False])
Random Sample Target: 9.382944107055664
